In [113]:
import torch
import pyro
import pandas as pd
import altair as alt
import seaborn as sns
import numpy as np
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.optim import Adam

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [53]:
df = pd.read_hdf("data/watch_minutes.hdf")

In [54]:
df

,user_id,video_id,watched_minutes
0,0,0,1
1,0,39,3
2,0,52,27
3,0,53,85
4,0,62,88
...,...,...,...
29306,12623,116,1
29307,12624,119,32
29308,12625,119,6
29309,12626,53,16


In [85]:
users.shape

(12628, 2)

In [88]:
users = df.groupby('user_id', as_index=False).agg({'video_id': len})
users
# sns.kdeplot(users['video_id'])

,user_id,video_id
0,0,38
1,1,2
2,2,5
3,3,3
4,4,3
...,...,...
12623,12623,1
12624,12624,1
12625,12625,1
12626,12626,1


In [58]:
pivot = df.pivot(index="user_id", columns="video_id", values="watched_minutes")

In [97]:
pivot.head()

video_id,0,1,2,3,4,5,6,7,8,9,...,555,556,557,558,559,560,561,562,563,564
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
pd.notna(pivot).sum(axis=1).sort_values(ascending=False)

user_id
10       179
142      101
1442      63
1432      46
1484      44
        ... 
2831       1
7345       1
2832       1
2833       1
12627      1
Length: 12628, dtype: int64

In [73]:
1 - np.sum(np.sum(pd.isna(pivot))) / (pivot.shape[0] * pivot.shape[1])

0.004108162504450008

In [106]:
def split_train_test(data, percent_test=0.1):

    n, m = data.shape
    N = n * m

   
    train = data.copy()
    test = np.ones(data.shape) * np.nan

   
    tosample = np.where(~np.isnan(train)) 
    idx_pairs = list(zip(tosample[0], tosample[1])) 

    test_size = int(len(idx_pairs) * percent_test)  
    train_size = len(idx_pairs) - test_size 

    indices = np.arange(len(idx_pairs))
    sample = np.random.choice(indices, replace=False, size=test_size)

    
    for idx in sample:
        idx_pair = idx_pairs[idx]
        test[idx_pair] = train[idx_pair] 
        train[idx_pair] = np.nan

   
    assert train_size == N - np.isnan(train).sum()
    assert test_size == N - np.isnan(test).sum()

    return train, test

In [108]:
data = pivot.values

In [110]:
train, test = split_train_test(data)

In [112]:
 def model(alpha, dim, n, m, nan_mask, not_na, data):
        """
        Perform matrix factorization
        R = U @ V.T
        """
        alpha_loc = torch.tensor(1 / 25)

        loc_u = pyro.sample(
            "loc_u",
            dist.MultivariateNormal(
                loc=torch.zeros(dim),
                precision_matrix=torch.eye(dim) * alpha_loc,
            ),
        )
        precission_u = pyro.sample(
            "precission_u",
            dist.LKJCorrCholesky(
                d=dim, eta=torch.tensor(alpha)
            ),
        )

        observations_scale = pyro.sample(
            "obs_scale",
            dist.InverseGamma(
                concentration=torch.tensor(1.0),
                rate=torch.tensor(1.0),
            ),
        )

        with pyro.plate("users", n):
            U = pyro.sample(
                "U", dist.MultivariateNormal(loc=loc_u, precision_matrix=precission_u)
            )
        with pyro.plate("content", m):
            V = pyro.sample(
                "V", dist.MultivariateNormal(loc=torch.zeros(dim), precision_matrix=torch.eye(dim))
            )
        with pyro.plate("observations", not_na):
            R = pyro.sample(
                "R",
                dist.Normal(loc=(U @ V.T)[~nan_mask], scale=observations_scale),
                obs=data,
            )


In [115]:
nan_mask = np.isnan(train)
not_na = (~nan_mask).sum()
data = torch.from_numpy(train[~nan_mask])


In [119]:
pyro.clear_param_store()
guide = AutoDiagonalNormal(model)
svi = SVI(model, guide, Adam({"lr": 0.001}), loss=Trace_ELBO())
n, m = train.shape
dim = 5
alpha = 2.0
iterations = 3000
train_loss = []
for i in range(iterations):
    loss = svi.step(alpha, dim, n, m, nan_mask, not_na, data)
    train_loss.append(loss / len(data))
    if i % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (i + 1, loss / len(data)))

[iteration 0001] loss: 619.1887
[iteration 0101] loss: 440.4312
[iteration 0201] loss: 351.5600
[iteration 0301] loss: 248.8039
[iteration 0401] loss: 179.5413
[iteration 0501] loss: 181.7405
[iteration 0601] loss: 207.1990
[iteration 0701] loss: 167.6693
[iteration 0801] loss: 151.5515
[iteration 0901] loss: 122.1684
[iteration 1001] loss: 95.4874
[iteration 1101] loss: 125.1443
[iteration 1201] loss: 73.0651
[iteration 1301] loss: 72.6004
[iteration 1401] loss: 79.3800
[iteration 1501] loss: 79.7348
[iteration 1601] loss: 58.7451
[iteration 1701] loss: 50.0347
[iteration 1801] loss: 47.5605
[iteration 1901] loss: 29.4954
[iteration 2001] loss: 42.6831
[iteration 2101] loss: 45.0082
[iteration 2201] loss: 31.5972
[iteration 2301] loss: 32.1334
[iteration 2401] loss: 28.4280
[iteration 2501] loss: 29.5484
[iteration 2601] loss: 25.9064
[iteration 2701] loss: 22.3851
[iteration 2801] loss: 18.0146
[iteration 2901] loss: 19.0082


torch.Size([65981])

In [137]:
guide.median()['loc_u']

tensor([ 1.4569, -0.1452,  1.6633,  1.3566, -1.1854], grad_fn=<ViewBackward>)

In [138]:
guide.median()['precission_u']

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9487,  0.3163,  0.0000,  0.0000,  0.0000],
        [-0.7101,  0.1080,  0.6957,  0.0000,  0.0000],
        [-0.6194,  0.3743, -0.4679,  0.5073,  0.0000],
        [ 0.5166, -0.2569,  0.3558,  0.5965,  0.4299]], grad_fn=<CopySlices>)

In [141]:
U_pooled = dist.MultivariateNormal(loc = guide.median()['loc_u'], precision_matrix=guide.median()['precission_u'])

In [144]:
U_pooled.sample()

tensor([ 0.7973, -1.2195,  0.8361,  0.6614, -1.0171])